In [1]:
import pandas as pd

In [2]:
def clear_leading_data(df):
    # Detect where the timer is reset by finding the first negative difference
    reset_indices = df[df['Timestamp'].diff() < 0].index

    df_cleaned = df
    if not reset_indices.empty:
        if(reset_indices[0] < 255):
            # Drop rows before the timer reset
            reset_index = reset_indices[0]
            df_cleaned = df.iloc[reset_index:].reset_index(drop=True)
    
    return df_cleaned

In [3]:
def correct_timestamps(df):
    
    # Correct the timestamp by detecting wraparound and adjusting accordingly
    max_timestamp_value = 65535  # Maximum value before wraparound

    # Create a corrected timestamp column
    df['Corrected_Timestamp'] = df['Timestamp'].copy()

    # Iterate over the DataFrame and correct the timestamps
    correction = 0
    for i in range(1, len(df)):
        if df.loc[i, 'Timestamp'] < df.loc[i - 1, 'Timestamp']:
            # Detected wraparound
            correction += max_timestamp_value
        df.loc[i, 'Corrected_Timestamp'] += correction
    
    return df

In [8]:
# Load the data into a pandas DataFrame from the CSV file
csv_file_path1 = "../output/a5_launch_0.csv"
csv_file_path2 = "../output/a5_launch_1.csv"

df1 = pd.read_csv(csv_file_path1)

df2 = pd.read_csv(csv_file_path2)

# Print the number of rows in each dataframe
print("Number of rows in df1: ", len(df1))
print("Number of rows in df2: ", len(df2))

Number of rows in df1:  786420
Number of rows in df2:  786420


In [ ]:
# Print the labels of all the dataframe columns

print("Columns in df1: ", df1.columns)

# Drop the columns Corrected_Timestamp, New Timestamp', 'Corrected_Difference', 'Previous_Timestamp'
df1 = df1.drop(columns=['Corrected_Timestamp', 'New Timestamp', 'Corrected_Difference', 'Previous_Timestamp'])

print("Columns in df1: ", df1.columns)


In [9]:
def find_first_difference(df1, df2):
    # Ensure that both DataFrames have the same shape
    if df1.shape != df2.shape:
        print("DataFrames have different shapes, cannot compare.")
        return

    # Compare the DataFrames element-wise
    differences = (df1 != df2)

    # Find the first row where there is at least one difference
    differing_rows = differences.any(axis=1)

    # Get the index of the first row where the two DataFrames differ
    first_diff_index = differing_rows.idxmax() if differing_rows.any() else None

    if first_diff_index is not None:
        print("First difference found at row index:", first_diff_index)
        print("df1 row:\n", df1.iloc[first_diff_index])
        print("df2 row:\n", df2.iloc[first_diff_index])
    else:
        print("No differences found between the two DataFrames.")

# Find and print the first row where the two DataFrames differ
find_first_difference(df1, df2)

First difference found at row index: 786408
df1 row:
 Timestamp              65535
X_Accel               -0.049
Y_Accel               -0.049
Z_Accel               -0.049
Pressure0      -21398.368072
Temperature0      190.196023
Humidity0              100.0
Pressure1      -14096.935409
Temperature1      190.266227
Humidity1              100.0
State                      7
StatusCode              0x1F
Name: 786408, dtype: object
df2 row:
 Timestamp              57799
X_Accel               -0.049
Y_Accel                0.784
Z_Accel               -0.784
Pressure0       88352.817582
Temperature0       29.952821
Humidity0          18.497084
Pressure1       97369.397951
Temperature1       28.124353
Humidity1          19.409711
State                      1
StatusCode              0x00
Name: 786408, dtype: object


In [10]:
def compare_differing_rows(df1, df2):
    # Ensure both DataFrames have the same shape
    if df1.shape != df2.shape:
        print("DataFrames have different shapes, cannot compare.")
        return
    
    # Compare the DataFrames element-wise
    differences = (df1 != df2)

    # Find rows where there is at least one difference
    differing_rows = differences.any(axis=1)

    # Get the indices of the differing rows
    differing_indices = differing_rows[differing_rows].index

    # Print out the differences
    if differing_indices.empty:
        print("No differences found.")
    else:
        print(f"Total number of differing rows: {len(differing_indices)}")  # Print the total count
        for index in differing_indices:
            print(f"Difference found at row index {index}:")
            # Print the entire row for both df1 and df2
            # print("df1 row:")
            # print(df1.iloc[index])
            # print("df2 row:")
            # print(df2.iloc[index])
            # Print the specific columns that are different
            differing_columns = differences.loc[index, differences.loc[index] == True].index
            for col in differing_columns:
                print(f"Column '{col}' differs: df1 = {df1.loc[index, col]}, df2 = {df2.loc[index, col]}")
            print("-" * 40)  # Divider between row comparisons
    
# Count and print the number of differing rows
compare_differing_rows(df1, df2)

Total number of differing rows: 12
Difference found at row index 786408:
Column 'Timestamp' differs: df1 = 65535, df2 = 57799
Column 'Y_Accel' differs: df1 = -0.049, df2 = 0.784
Column 'Z_Accel' differs: df1 = -0.049, df2 = -0.784
Column 'Pressure0' differs: df1 = -21398.368072, df2 = 88352.817582
Column 'Temperature0' differs: df1 = 190.196023, df2 = 29.952821
Column 'Humidity0' differs: df1 = 100.0, df2 = 18.497084
Column 'Pressure1' differs: df1 = -14096.935409, df2 = 97369.397951
Column 'Temperature1' differs: df1 = 190.266227, df2 = 28.124353
Column 'Humidity1' differs: df1 = 100.0, df2 = 19.409711
Column 'State' differs: df1 = 7, df2 = 1
Column 'StatusCode' differs: df1 = 0x1F, df2 = 0x00
----------------------------------------
Difference found at row index 786409:
Column 'Timestamp' differs: df1 = 65535, df2 = 57810
Column 'X_Accel' differs: df1 = -0.049, df2 = -0.098
Column 'Y_Accel' differs: df1 = -0.049, df2 = 0.784
Column 'Z_Accel' differs: df1 = -0.049, df2 = -0.882
Column

In [11]:
def drop_differing_rows(df1, df2):
    # Ensure both DataFrames have the same shape
    if df1.shape != df2.shape:
        print("DataFrames have different shapes, cannot compare.")
        return df1, df2
    
    # Compare the DataFrames element-wise
    differences = (df1 != df2)

    # Find rows where there is at least one difference
    differing_rows = differences.any(axis=1)

    # Get the indices of the differing rows
    differing_indices = differing_rows[differing_rows].index

    # If there are differing rows, drop them
    if not differing_indices.empty:
        df1_cleaned = df1.drop(index=differing_indices)
        df2_cleaned = df2.drop(index=differing_indices)
        print(f"Dropped {len(differing_indices)} differing rows.")
    else:
        df1_cleaned = df1
        df2_cleaned = df2
        print("No differences found.")

    return df1_cleaned, df2_cleaned

df1, df2 = drop_differing_rows(df1, df2)

# Print the number of total rows in the dataframes
print(f"df1 rows: {len(df1)}")
print(f"df2 rows: {len(df2)}")

Dropped 12 differing rows.
df1 rows: 786408
df2 rows: 786408


In [12]:
def add_new_timestamp_column(df):
    # Create a new column "New Timestamp" with values starting from 0, incrementing by 20
    df['New Timestamp'] = range(0, 20 * len(df), 20)
    return df

# Correct Timestamps
df1 = clear_leading_data(df1)
df1 = correct_timestamps(df1)

# df2 = clear_leading_data(df2)
# df2 = correct_timestamps(df2)

df1 = add_new_timestamp_column(df1)
# df2 = add_new_timestamp_column(df2)

In [13]:
def find_inconsistent_increment(df):
    # Calculate the differences between consecutive corrected timestamps
    df['Corrected_Difference'] = df['Corrected_Timestamp'].diff()

    # Find the rows where the difference is not between 10 and 30
    inconsistent_rows = df[(df['Corrected_Difference'] < 5) | (df['Corrected_Difference'] > 15)]
    
    print(len(inconsistent_rows))

    # if not inconsistent_rows.empty:
    #     # Get the indices and corresponding differences of inconsistent increments
    #     inconsistent_indices = inconsistent_rows.index
    #     num_inconsistencies = len(inconsistent_indices)
        
    #     print(f"Total number of inconsistent increments: {num_inconsistencies}")
    #     print("\nDetails of inconsistent increments:")
        
    #     for idx in inconsistent_indices:
    #         # Print the current row and its previous rows up to 2 rows back
    #         print(f"Inconsistency at row index: {idx}")
    #         start_index = max(0, idx - 2)
    #         # Print the details of the current and previous rows (up to 2 rows back)
    #         for i in range(start_index, idx + 1):
    #             timestamp = df.iloc[i]['Corrected_Timestamp']
    #             difference = df.iloc[i]['Corrected_Difference'] if i > 0 else 'N/A'
    #             print(f"Row index: {i}, Timestamp: {timestamp}, Difference to last timestamp: {difference}")
    #         print("-" * 40)
    # else:
    #     print("No inconsistent increments found.")
        

find_inconsistent_increment(df1)

2


In [14]:
def count_timestamps_in_range(df, lower_bound=0, upper_bound=10, previous_value_threshold=65500):
    # Shift the "Timestamp" column to compare with previous row
    df['Previous_Timestamp'] = df['Timestamp'].shift(1)
    
    # Filter rows where the "Timestamp" is between lower_bound and upper_bound
    # and the previous value is less than previous_value_threshold
    condition = (
        (df['Timestamp'] >= lower_bound) & 
        (df['Timestamp'] <= upper_bound) & 
        (df['Previous_Timestamp'] < previous_value_threshold)
    )
    
    # print only the first 10 rows and only the 'Timestamp' and 'Previous_Timestamp' columns
    # print(df[condition].tail(30)[['Timestamp', 'Previous_Timestamp']])
    
    count = df[condition].shape[0]

    print(f"Number of occurrences where 'Timestamp' is between {lower_bound} and {upper_bound} "
          f"and the previous value is less than {previous_value_threshold}: {count}")

    
count_timestamps_in_range(df1, previous_value_threshold=65500)

Number of occurrences where 'Timestamp' is between 0 and 10 and the previous value is less than 65500: 16


In [15]:
# Find the number of occurances where the status code is not "0x00"
def count_status_code(df, StatusCode='0x00'):
    # Filter rows where the 'Status' column is not equal to status_code
    condition = (df['StatusCode'] != StatusCode)
    
    # Count the number of rows that meet the condition
    count = df[condition].shape[0]
    
    print(f"Number of occurrences where 'Status' is not '{StatusCode}': {count}")
    
count_status_code(df1, StatusCode='0x00')

Number of occurrences where 'Status' is not '0x00': 0


In [18]:
# Drop any rows in which the status code is not '0x00'
def drop_status_code(df, StatusCode='0x00'):
    # Filter rows where the 'Status' column is equal to status_code
    condition = (df['StatusCode'] == StatusCode)
    
    # Drop rows that meet the condition
    df_cleaned = df[condition].reset_index(drop=True)
    
    return df_cleaned

df1 = drop_status_code(df1, StatusCode='0x00')

In [16]:
import matplotlib.pyplot as plt
import os

def plot_acceleration_vs_time(df, output_dir):

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(df['New Timestamp'], df['X_Accel'], label='X Accel')
    plt.plot(df['New Timestamp'], df['Y_Accel'], label='Y Accel')
    plt.plot(df['New Timestamp'], df['Z_Accel'], label='Z Accel')
    plt.xlabel('Timestamp')
    plt.ylabel('Acceleration (Gs)')
    plt.title('Acceleration on X, Y, and Z vs Time')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'acceleration_vs_time1.png'))
    plt.close()

# Example usage
output_dir = "../graphs"
plot_acceleration_vs_time(df1, output_dir)

In [23]:
def plot_filtered_acceleration(df, output_dir, start_time, end_time):
    # Filter the DataFrame to include only rows within the specified timestamp range
    filtered_df = df[(df['New Timestamp'] >= start_time) & (df['New Timestamp'] <= end_time)]
    
    # Convert 'New Timestamp' from milliseconds to seconds
    filtered_df['New Timestamp (s)'] = filtered_df['New Timestamp'] / 1000
    
    # Plot the filtered data
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_df['New Timestamp (s)'], filtered_df['X_Accel'], label='X Accel')
    plt.plot(filtered_df['New Timestamp (s)'], filtered_df['Y_Accel'], label='Y Accel')
    plt.plot(filtered_df['New Timestamp (s)'], filtered_df['Z_Accel'], label='Z Accel')
    plt.xlabel('New Timestamp (seconds)')
    plt.ylabel('Acceleration (Gs)')
    plt.title('Acceleration on X, Y, and Z vs Corrected Timestamp (in seconds)')
    plt.legend()
    plt.grid(True)
    
    # Save the plot
    plt.savefig(os.path.join(output_dir, 'filtered_acceleration_vs_time1.png'))
    plt.close()

# Example usage
output_dir = "../graphs"
start_time = 1.522e7
end_time = 1.533e7
plot_filtered_acceleration(df1, output_dir, start_time, end_time)


/tmp/ipykernel_581228/277254526.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['New Timestamp (s)'] = filtered_df['New Timestamp'] / 1000


In [24]:
# write a function to print a dataframe to CSV
def write_to_csv(df, output_path):
    df.to_csv(output_path, index=False)
    
# Example usage
output_path = "../output/a5.csv"
write_to_csv(df1, output_path)

In [25]:
import matplotlib.pyplot as plt
import os

def plot_temperature_vs_time(df, output_dir):
    # Convert 'New Timestamp' from milliseconds to seconds
    df['New Timestamp (s)'] = df['New Timestamp'] / 1000

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(df['New Timestamp (s)'], df['Temperature0'], label='Temp0')
    plt.plot(df['New Timestamp (s)'], df['Temperature1'], label='Temp1')
    plt.xlabel('Timestamp')
    plt.ylabel('Temperature (C)')
    plt.title('Temperature0 and Temperature1 vs Time')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'temperature_vs_time1.png'))
    plt.close()

# Example usage
output_dir = "../graphs"
plot_temperature_vs_time(df1, output_dir)

In [17]:
import matplotlib.pyplot as plt
import os

def plot_pressure_vs_time(df, output_dir):
    # Convert 'New Timestamp' from milliseconds to seconds
    df['New Timestamp (s)'] = df['New Timestamp'] / 1000

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(df['New Timestamp (s)'], df['Pressure0'], label='Press0')
    plt.plot(df['New Timestamp (s)'], df['Pressure1'], label='Press1')
    plt.xlabel('Timestamp')
    plt.ylabel('Pressure (Pa)')
    plt.title('Pressure0 and Pressure1 vs Time')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'pressure_vs_time1.png'))
    plt.close()

# Example usage
output_dir = "../graphs"
plot_pressure_vs_time(df1, output_dir)

In [18]:
import matplotlib.pyplot as plt
import os

def plot_humidity_vs_time(df, output_dir):
    # Convert 'New Timestamp' from milliseconds to seconds
    df['New Timestamp (s)'] = df['New Timestamp'] / 1000

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(df['New Timestamp (s)'], df['Humidity0'], label='Hum0')
    plt.plot(df['New Timestamp (s)'], df['Humidity1'], label='Hum1')
    plt.xlabel('Timestamp')
    plt.ylabel('Humidity (%)')
    plt.title('Humidity0 and Humidity1 vs Time')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'humidity_vs_time1.png'))
    plt.close()

# Example usage
output_dir = "../graphs"
plot_humidity_vs_time(df1, output_dir)

In [43]:
import matplotlib.pyplot as plt
import os

def plot_acceleration_vs_time(df, output_dir, val, pre_event_seconds=150, post_event_minutes=45):
    # Calculate the event condition
    df['X_Accel_Abs'] = df['X_Accel'].abs()
    df['Y_Accel_Abs'] = df['Y_Accel'].abs()
    df['Z_Accel_Abs'] = df['Z_Accel'].abs()
    
    # Find the index of the first row where any of the absolute accelerometer values exceeds val
    condition = (df['X_Accel_Abs'] > val) | (df['Y_Accel_Abs'] > val) | (df['Z_Accel_Abs'] > val)
    event_index = df[condition].index
    
    if event_index.empty:
        print("No events found with accelerometer values exceeding the specified threshold.")
        return
    
    event_index = event_index[0]
    event_time = df.loc[event_index, 'Corrected_Timestamp']
    
    # Define the time range for plotting
    start_time = event_time - (pre_event_seconds * 1000)  # Convert seconds to milliseconds
    end_time = event_time + (post_event_minutes * 60 * 1000)  # Convert minutes to milliseconds
    
    # Filter the DataFrame to include only rows within the specified time range
    filtered_df = df[(df['Corrected_Timestamp'] >= start_time) & (df['Corrected_Timestamp'] <= end_time)]
    
    # Plot the filtered data
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_df['Corrected_Timestamp'], filtered_df['X_Accel'], label='X Accel')
    plt.plot(filtered_df['Corrected_Timestamp'], filtered_df['Y_Accel'], label='Y Accel')
    plt.plot(filtered_df['Corrected_Timestamp'], filtered_df['Z_Accel'], label='Z Accel')
    plt.xlabel('Timestamp')
    plt.ylabel('Acceleration (Gs)')
    plt.title('Acceleration on X, Y, and Z vs Time')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'acceleration_vs_time_filtered.png'))
    plt.close()

# Example usage
output_dir = "../graphs"
val = 5
plot_acceleration_vs_time(df1, output_dir, val)

In [20]:
def plot_filtered_acceleration(df, output_dir, start_time, end_time):
    # Filter the DataFrame to include only rows within the specified timestamp range
    filtered_df = df[(df['Corrected_Timestamp'] >= start_time) & (df['Corrected_Timestamp'] <= end_time)]
    
    # Plot the filtered data
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_df['Corrected_Timestamp'], filtered_df['X_Accel'], label='X Accel')
    plt.plot(filtered_df['Corrected_Timestamp'], filtered_df['Y_Accel'], label='Y Accel')
    plt.plot(filtered_df['Corrected_Timestamp'], filtered_df['Z_Accel'], label='Z Accel')
    plt.xlabel('Corrected Timestamp')
    plt.ylabel('Acceleration (Gs)')
    plt.title('Acceleration on X, Y, and Z vs Corrected Timestamp')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'filtered_acceleration_vs_time.png'))
    plt.close()

# Example usage
output_dir = "../graphs"
start_time = 9.245e8
end_time = 9.247e8
plot_filtered_acceleration(df2, output_dir, start_time, end_time)